In [ ]:
from pathlib import Path

import pandas as pd

from blooms_ml.utils_ferrybox import get_dataframe_ferrybox2002to2018, get_ferrytracks, plot_temp_salt_flu

In [ ]:
VARIABLES = [*["timestamps"], *[f"temp_{i}" for i in range(64)], *[f"salt_{i}" for i in range(64)]]

In [ ]:
datadir = f"{Path.home()}/data_ferrybox"
dfs = get_ferrytracks(datadir)

In [ ]:
df_labels = get_dataframe_ferrybox2002to2018(dfs)

In [ ]:
df_labels

In [ ]:
def to_differences(df):
    df = df.reset_index(drop=True)
    df_diff1 = (df[VARIABLES].diff(periods=1))[2:]
    df_diff1 = df_diff1.rename(columns=lambda x: x + "_diff1")
    df_diff2 = (df[VARIABLES].diff(periods=2))[2:]
    df_diff2 = df_diff2.rename(columns=lambda x: x + "_diff2")
    df_diff = pd.concat([df_diff1, df_diff2], axis=1)
    df_diff[["timestamps", "labels"]] = df[["timestamps", "labels"]]
    return df_diff

In [ ]:
df_labels_diff = to_differences(df_labels)
df_labels_diff = df_labels_diff[df_labels_diff["timestamps_diff2"] < pd.Timedelta(days=7)]

In [ ]:
blooms = df_labels[df_labels["labels"] == 1]

In [ ]:
blooms

In [ ]:
snippet = dfs[100].copy()
snippet.set_index("Time", inplace=True)

In [ ]:
print(f"Start date: {snippet.index[0]}")
print(f"End date: {snippet.index[-1]}")
print(f"Length: {len(snippet)}")
plot_temp_salt_flu(snippet)

PCA

In [ ]:
import time

from sklearn.decomposition import PCA

In [ ]:
df_labels_diff

In [ ]:
X = df_labels_diff.drop(["timestamps_diff1", "timestamps_diff2", "timestamps", "labels"], axis=1)
y = df_labels_diff["labels"]

In [ ]:
# PCA
t0 = time.time()
pca = PCA(n_components=10, random_state=42)
X_reduced_pca = pca.fit_transform(X.values)
t1 = time.time()
print(f"PCA took {t1 - t0:.2} s")

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
x1, x2, x3 = X_reduced_pca[::3, 0], X_reduced_pca[::3, 1], X_reduced_pca[::3, 2]

In [ ]:
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

In [ ]:
# Create a 3D scatter plot
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

blue_patch = mpatches.Patch(color="#0A0AFF", label="No bloom")
red_patch = mpatches.Patch(color="#AF0000", label="bloom")

# Scatter plot
ax.scatter(x1, x2, x3, c=(y[::3] == 0), cmap="coolwarm", label="No bloom", linewidths=2)
ax.scatter(x1, x2, x3, c=(y[::3] == 1), cmap="coolwarm", label="bloom", linewidths=2)

ax.grid(True)
ax.legend(handles=[blue_patch, red_patch])
plt.show()

Classification

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
classifier = DecisionTreeClassifier()

In [ ]:
classifier.fit(X_train, y_train)

In [ ]:
training_score = cross_val_score(classifier, X_test, y_test, cv=5)

In [ ]:
print(
    "Classifier: ",
    classifier.__class__.__name__,
    "Has a training score of",
    round(training_score.mean(), 2) * 100,
    "% accuracy score",
)